In [99]:
import pandas as pd

file_path = "netflix_titles.csv"

pd.options.mode.chained_assignment = None

data = pd.read_csv(file_path,sep=",",header=0,encoding="utf-8",index_col=False)
data = data.dropna()

def create_new_csv_no(name, data):
    data.to_csv(name, sep=",", encoding="utf-8",index=False)
def create_new_csv(name, data):
    data.to_csv(name, sep=",", encoding="utf-8")


create_new_csv_no("data.csv", data)




In [100]:
movie = data[['show_id','title', 'description','duration','type']]
movie= movie[movie['type'] == 'Movie']
movie = movie.drop(columns=['type'])
movie['duration'] = movie['duration'].str.extract('(\d+)', expand=False).astype(int)
create_new_csv_no("movie.csv", movie)

In [101]:
series = data[['show_id','title', 'description','duration','type']]
series= series[series['type'] == 'TV Show']
series = series.drop(columns=['type'])
series['duration'] = series['duration'].str.extract('(\d+)', expand=False).astype(int)
create_new_csv_no("series.csv", series)


In [102]:
person1 = pd.DataFrame(data['cast'].str.split(',').explode().str.strip())
person1.columns = ['person']
person2 = pd.DataFrame(data['director'].str.split(',').explode().str.strip())
person2.columns = ['person']

person = pd.concat([person1, person2])
person.drop_duplicates(subset="person", keep='first', inplace=True)
person['no_id'] = person.reset_index().index
create_new_csv_no("person.csv", person)

In [103]:
date = data[['date_added']]
date.drop_duplicates(subset = 'date_added', keep = 'first', inplace = True)
date['no_date'] = date.reset_index().index
create_new_csv_no("date.csv", date)

In [104]:
country = pd.DataFrame(data['country'].str.split(',').explode().str.strip())
country.drop_duplicates(subset = 'country', keep = 'first', inplace = True)
country['no_country'] = country.reset_index().index
create_new_csv_no("country.csv", country)

In [105]:
rating = pd.DataFrame(data['rating'].str.split(',').explode().str.strip())
rating.drop_duplicates(subset = 'rating', keep = 'first', inplace = True)
rating['no_rating'] = rating.reset_index().index
create_new_csv_no("rating.csv", rating)

In [106]:
genre = pd.DataFrame(data['listed_in'].str.split(',').explode().str.strip())
genre.drop_duplicates(subset = 'listed_in', keep = 'first', inplace = True)
genre['no_genre'] = genre.reset_index().index

create_new_csv_no("genre.csv", genre)

In [115]:
#Actor.csv
find_actor = data[['show_id','cast']]
# Decompose the column "cast" into different names of persons with show_id
cast = find_actor.copy()
cast['cast'] = cast['cast'].str.split(', ')
cast= cast.explode('cast')

person= pd.read_csv("person.csv",sep=",",header=0,encoding="utf-8",index_col=False)

cast = cast.merge(person, left_on='cast', right_on='person', how='left')
cast = cast[['show_id', 'no_id']]
cast.columns = ['show_id', 'cast_id']
create_new_csv_no("actor.csv", cast)



In [116]:
#Director.csv

find_director = data[['show_id','director']]
# Decompose the column "director" into different names of persons with show_id
director = find_director.copy()
director['director'] = director['director'].str.split(', ')
director= director.explode('director')

director = director.merge(person, left_on='director', right_on='person', how='left')
director = director[['show_id', 'no_id']]
director.columns = ['show_id', 'director_id']
create_new_csv_no("director.csv", director)

In [117]:
#Produced.csv

find_produced = data[['show_id','country']]
# Decompose the column "country" into different names of countries with show_id
produced = find_produced.copy()
produced['country'] = produced['country'].str.split(', ')
produced= produced.explode('country')

country= pd.read_csv("country.csv",sep=",",header=0,encoding="utf-8",index_col=False)

produced = produced.merge(country, left_on='country', right_on='country', how='left')
produced = produced[['show_id', 'no_country']]
produced.columns = ['show_id', 'country_id']
create_new_csv_no("produced.csv", produced)

In [118]:
#Belong.csv

find_belong = data[['show_id','listed_in']]
# Decompose the column "listed_in" into different names of genres with show_id
belong = find_belong.copy()
belong['listed_in'] = belong['listed_in'].str.split(', ')
belong= belong.explode('listed_in')

genre= pd.read_csv("genre.csv",sep=",",header=0,encoding="utf-8",index_col=False)

belong = belong.merge(genre, left_on='listed_in', right_on='listed_in', how='left')
belong = belong[['show_id', 'no_genre']]
belong.columns = ['show_id', 'genre_id']
create_new_csv_no("belong.csv", belong)